In [1]:
data_var = '2023-12-09'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9459,2023-12-09,Romênia Divisão A,13:00,CSM Oradea,SCM Craiova,1.20,4.27,148.5,1.82,1.84,-11.5,2.05,1.69,EyvU04ob,0.833333,0.234192,0.549451,0.543478,0.067525,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,124.528,41.472751,0.333040,2.4,1.341641,0.559017,121.68,1.596,0.402902,0.252445,26.0,95,78,1.37,1.56,0.000,0.0,0.793718,0.007728,0.136128,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
9460,2023-12-09,República Tcheca Nbl,13:00,Opava,Slavia Prague,1.06,7.70,167.5,1.85,1.85,-16.5,2.06,1.68,IgMuZLFT,0.943396,0.129870,0.540541,0.540541,0.073266,135.170,42.512743,0.314513,3.0,0.000000,0.000000,125.61,1.476,0.388175,0.262991,28.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,79,93,1.59,2.76,0.000,0.0,1.071961,0.000000,0.143690,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
9461,2023-12-09,Romênia Divisão A,12:00,CSM Focsani,CSM Constanta,2.62,1.45,152.5,1.87,1.79,-3.5,2.00,1.72,WClP1pWi,0.381679,0.689655,0.534759,0.558659,0.071335,378.484,264.854091,0.699776,0.0,0.000000,NaN,530.58,5.216,3.452598,0.661924,-32.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,74,83,7.17,1.38,0.000,0.0,0.406543,0.030912,0.106446,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
9462,2023-12-09,Romênia Divisão A,12:00,Brasov,BC Timisoara,1.53,2.39,155.5,1.87,1.83,-5.5,2.09,1.66,SlHMqaqE,0.653595,0.418410,0.534759,0.546448,0.072005,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,68,101,8.90,1.03,0.000,0.0,0.310261,0.015289,0.162163,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
9463,2023-12-09,República Tcheca Nbl,14:00,Olomoucko,Decin,2.07,1.70,171.5,1.85,1.85,1.5,1.92,1.79,fFD7UuOp,0.483092,0.588235,0.540541,0.540541,0.071327,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,94,93,5.40,1.65,0.000,0.0,0.138795,0.000000,0.049555,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9799,2023-12-09,Eua Ncaa,17:30,Arkansas State,UAB,2.23,1.64,150.5,1.83,1.83,3.5,1.73,0.00,vBab9KdP,0.448430,0.609756,0.546448,0.546448,0.058187,517.738,568.661844,1.098358,0.0,0.000000,NaN,1235.00,7.586,8.332459,1.098400,-67.0,101.956,21.201464,0.207947,1.8,1.643168,0.912871,100.86,1.404,0.365691,0.260464,-14.0,65,82,19.00,1.23,0.000,0.0,0.215604,0.000000,1.414214,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
9800,2023-12-09,Eua Ncaa,17:15,Arizona,Wisconsin,1.30,3.47,145.5,1.91,1.91,-8.5,1.91,1.91,Sn8GfFKN,0.769231,0.288184,0.523560,0.523560,0.057415,116.230,40.659424,0.349819,3.0,0.000000,0.000000,83.64,1.378,0.601806,0.436724,115.0,129.020,41.545544,0.322009,3.0,0.000000,0.000000,172.90,1.822,0.610098,0.334851,66.0,82,70,1.02,2.47,0.000,0.0,0.643363,0.000000,0.000000,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
9801,2023-12-09,Eua Ncaa,17:00,St. Louis,Hofstra,2.47,1.55,151.5,1.91,1.91,3.5,1.91,1

## Apostas do dia

### Back Home V1 (QCut)

In [ ]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V2 (LR)

In [ ]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    
    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
8476,07:00,Israel Liga Leumit,Elitzur Shomron,Gilboa Galil,2.26,Back Home
8477,07:15,Israel Liga Leumit,Elitzor Netania,Ironi Nahariya,1.98,Back Home
8481,15:30,Portugal Proliga,Sangalhos,Braga,1.98,Back Home
8484,12:00,Portugal Lfb Feminina,Gdessa Barreiro F,Esgueira F,1.66,Back Home
8485,14:00,Hungria Nacional,Szolnoki Olaj,Kormend,1.73,Back Home
8488,16:00,França Pro B,Fos-sur-Mer,Orleans,1.98,Back Home
8492,16:30,França Pro B,St. Chamond,Boulazac,1.61,Back Home
8496,16:00,França Pro B,Nantes,Chalons-Reims,1.69,Back Home
8498,16:00,França Pro B,Denain-Voltaire,La Rochelle,2.29,Back Home
8499,16:00,França Pro B,Aix-Maurienne,Antibes,1.97,Back Home


### Under V1 (QCut)

In [ ]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO UNDER V1 (QCUT) ------------



,Time,League,Home,Away,Odds_Under,Bet
8546,15:00,Áustria Superliga,Kapfenberg,Swans Gmunden,1.79,Back Home
8547,13:30,Áustria Superliga,UBSC Graz,Vienna,1.85,Back Home
8595,11:30,Bahrein Premier League,Al-Hala,Al Najma,1.85,Back Home


### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9507,03:05,Japão B.League,Utsunomiya Brex,Brave Thunders,1.79,Back Home
9533,11:00,Lituânia Nkl,Kretinga,Vilkaviskis,2.15,Back Home
9549,18:30,Eua Ncaa,Texas A&M-CC,Nebraska O.,1.71,Back Home
9557,18:00,Eua Ncaa,Incarnate Word,Northern Arizona,2.23,Back Home
9574,16:00,Eua Ncaa,Vermont,Colgate,1.69,Back Home
9576,21:00,Eua Ncaa,South Dakota Coyotes,Cal St. Bakersfield,1.76,Back Home
9586,16:00,Eua Ncaa,South Carolina St,Bethune Cookman,1.71,Back Home
9638,02:00,Coreia Do Sul Kbl,Mobis Phoebus,Anyang JungKwanJang,1.72,Back Home
9735,23:00,Eua Ncaa,Pepperdine,UC San Diego,1.55,Back Home
9739,17:00,França Lnb,Cholet,JL Bourg,2.33,Back Home
